In [ ]:
import pandas as pd
from tqdm import tqdm
import os
import time

import sys
sys.path.append('../mlrose')
from mlrose_hiive.generators.queens_generator import QueensGenerator
from mlrose_hiive import SARunner
from mlrose_hiive import GeomDecay, ExpDecay, ArithDecay
import numpy as np

In [ ]:
ALGORITHM = 'sa'
PROBLEM_SIZE_LIST = [10, 30, 50, 100, 200]
# MAX_ITERATIONS = 10000
MAX_ATTEMPTS = 1000
NUM_RUNS = 3
MIN_T = 0.001

TEMPERATURE = 10
DECAY = 'ExpDecay'

In [ ]:
def get_max_size(problem_size):
    if problem_size <= 1:
        return 0
    if problem_size == 2:
        return 1
    return 3*(problem_size-2)

In [ ]:
P = 0.8
def get_exp_decay(init_temp, max_iter):
    r = (
        -np.log(MIN_T / init_temp)
        /
        int(P * max_iter)
    )
    return ExpDecay(init_temp=init_temp, exp_const=r, min_temp=MIN_T)
def get_geom_decay(init_temp, max_iter):
    r = (
        (MIN_T / init_temp) ** (1 / int(P * max_iter))
    )
    return GeomDecay(init_temp=init_temp, decay=r, min_temp=MIN_T)
def get_arith_decay(init_temp, max_iter):
    r = (
        (init_temp - MIN_T)
        /
        int(P * max_iter)
    )
    return ArithDecay(init_temp=init_temp, decay=r, min_temp=MIN_T)

In [ ]:
df_path = f'metrics_problem_size/{ALGORITHM}.csv'
os.makedirs(os.path.dirname(df_path), exist_ok=True)
if not os.path.exists(df_path):
    print('Creating new metrics file..')
    all_df = pd.DataFrame(columns=[
        # runner columns
        'Iteration', 'Time', 'Fitness', 'FEvals', 'Temperature', 'max_iters',
        # problem config
        'problem_size', 'decay_type', 'max_attempts', 
        # run config
        'run_number', 'group_timestamp', 'total_runtime',
    ])
    all_df.to_csv(df_path, index=False)
all_df = pd.read_csv(df_path)

In [ ]:
for problem_size in PROBLEM_SIZE_LIST:
    print(f"Problem Size: {problem_size}")

    MAX_ITERATIONS = 100 * problem_size

    if DECAY == 'ExpDecay':
        decay_type = get_exp_decay(TEMPERATURE, MAX_ITERATIONS)
    elif DECAY == 'GeomDecay':
        decay_type = get_geom_decay(TEMPERATURE, MAX_ITERATIONS)
    elif DECAY == 'ArithDecay':
        decay_type = get_arith_decay(TEMPERATURE, MAX_ITERATIONS)
    else:
        raise ValueError('Invalid decay type')

    if (
        (all_df['problem_size'] == problem_size) &
        (all_df['decay_type'] == DECAY) &
        (all_df['Temperature'] == TEMPERATURE) &
        (all_df['max_attempts'] == MAX_ATTEMPTS)
    ).any():
        print('Already run')
        continue

    group_timestamp = int(time.time())
    time.sleep(1)
    max_fitnesses = []
    for run_i in tqdm(range(NUM_RUNS)):
        t_start = time.perf_counter()
        problem = QueensGenerator().generate(seed=run_i, size=problem_size, maximize=True)
        runner = SARunner(
            problem=problem,
            experiment_name='SA',
            output_directory='.',
            seed=run_i,
            iteration_list=[MAX_ITERATIONS],
            max_attempts=MAX_ATTEMPTS,
            temperature_list=[decay_type],
            # decay_list=[decay_type],
        )
        _, df_run_curves = runner.run()

        df_run_curves['problem_size'] = problem_size
        df_run_curves['decay_type'] = DECAY
        df_run_curves['max_attempts'] = MAX_ATTEMPTS

        df_run_curves['Temperature'] = TEMPERATURE
        df_run_curves['max_iters'] = MAX_ITERATIONS

        df_run_curves['run_number'] = run_i
        df_run_curves['group_timestamp'] = group_timestamp

        max_fitnesses.append(df_run_curves['Fitness'].max())

        t_end = time.perf_counter()
        df_run_curves['total_runtime'] = t_end - t_start
        df_run_curves['optimal_fitness'] = get_max_size(problem_size)

        all_df = pd.concat([all_df, df_run_curves])

    print(f'Max Fitness: {np.mean(max_fitnesses)}')
    all_df.reset_index(inplace=True, drop=True)
    all_df.to_csv(df_path, index=False)

In [ ]:
all_df.columns

In [ ]:
agg_df = pd.DataFrame()
for group_timestamp in all_df['group_timestamp'].unique():
    df = all_df[all_df['group_timestamp'] == group_timestamp]
    max_fit = df['Fitness'].max()
    total_iters = df['Iteration'].max()
    max_fevals = df['FEvals'].max()
    df.drop(columns=['Fitness', 'Iteration', 'FEvals', 'Time'], inplace=True)
    assert(len(df) > NUM_RUNS)
    df.drop_duplicates(inplace=True)
    assert(len(df) == NUM_RUNS)
    df['max_fitness'] = max_fit
    df['total_iterations'] = total_iters
    df['max_fevals'] = max_fevals
    agg_df = pd.concat([agg_df, df])
agg_df.reset_index(inplace=True, drop=True)

In [ ]:
df_path = f'metrics_problem_size/{ALGORITHM}_agg.csv'
agg_df.to_csv(df_path, index=False)

In [ ]:
agg_df